In [0]:
%pip install -r ../requirements.txt

In [0]:
dbutils.widgets.text("region", defaultValue="eu-west-1")
dbutils.widgets.text("catalog", defaultValue="production")
dbutils.widgets.text("uc_service_credential", defaultValue="production-aws-secrets-manager")
dbutils.widgets.text("secrets_manager_arn", defaultValue="")

metastore_id = sql("SELECT element_at(split(current_metastore(), ':'), -1) AS metastore").first().metastore
catalog = dbutils.widgets.get("catalog")
region = dbutils.widgets.get("region")
service_credential = dbutils.widgets.get("uc_service_credential")
secrets_manager_arn = dbutils.widgets.get("secrets_manager_arn")

In [0]:
import boto3

crypto_functions = dbutils.import_notebook("notebooks.envelope_encryption_v2.common.crypto_functions")

boto3_session = boto3.Session(botocore_session=dbutils.credentials.getServiceCredentialsProvider(service_credential), region_name=dbutils.widgets.get("region"))
secrets = crypto_functions.get_aws_secret(boto3_session, secrets_manager_arn)
prev_kek_password, prev_kek_salt = secrets.pop("kek_password"), secrets.pop("kek_salt")

In [0]:
new_kek = crypto_functions.generate_kek()

rotated_secrets = {}

for k, v in secrets.items():

    if k.endswith("_dek"):
        new_dek = {}
        schema = k.split('_dek')[0]
        print(f"Rotating DEK for schema '{schema}'...")
        dek, nonce, tag = secrets.get(f"{schema}_dek"), secrets.get(f"{schema}_nonce"), secrets.get(f"{schema}_tag")
        encrypted_dek = crypto_functions.encrypt_with_kek(kek_password=new_kek.get("kek_password"), kek_salt=new_kek.get("kek_salt"), to_encrypt=crypto_functions.decrypt_with_kek(kek_password=prev_kek_password, kek_salt=prev_kek_salt, to_decrypt=dek, nonce=nonce, tag=tag))
        new_dek.update({f"{schema}_dek": encrypted_dek.pop("encrypted_string"),
            f"{schema}_nonce": encrypted_dek.pop("nonce"),
            f"{schema}_tag": encrypted_dek.pop("tag")})
        rotated_secrets.update(new_dek)

In [0]:
import json

response = crypto_functions.put_aws_secret(boto3_session, secrets_manager_arn, json.dumps(new_kek|rotated_secrets))
response